In [2]:
#CTE de mejores vendedores, vendedores que mas generan ingresos
from src.db import run_sql
query= """WITH vendedor_top as (
		SELECT          SalesPersonID,
                        SUM(TotalPrice) as Ingreso_Generado
                        FROM sales
                        GROUP BY SalesPersonID
                        )
	 SELECT 
			vt.SalesPersonID,
            vt.Ingreso_Generado,
            e.FirstName,
            e.LastName
	 FROM vendedor_top AS vt
     JOIN employees as e
     ON vt.SalesPersonID = e.EmployeeID
     ORDER BY Ingreso_Generado DESC
     LIMIT 5;"""

df_customers = run_sql(query)
df_customers 

,SalesPersonID,Ingreso_Generado,FirstName,LastName
0,23,665091.0,Janet,Flowers
1,22,631158.0,Tonia,Mc Millan
2,21,618744.0,Devon,Brewer
3,20,569220.0,Shelby,Riddle
4,19,538954.0,Bernard,Moody


In [ ]:
#VISTA QUE INDICA EL INGRESO POR MES
from src.db import run_sql
query= """ 
SELECT * FROM ventas_total_mes
ORDER BY Mes ASC;"""

df_customers = run_sql(query)
df_customers 

,Año,Mes,TotalMensual
0,2025,1,708878.0
1,2025,2,633065.0
2,2025,3,742401.0
3,2025,4,692481.0
4,2025,5,710675.0
5,2025,6,729443.0
6,2025,7,728310.0
7,2025,8,738776.0
8,2025,9,718757.0
9,2025,10,731963.0


In [ ]:
#Funcion de ventana, RowNum que asigna un número secuencial a cada fila, ordenadas primero por Año y luego por Mes
from src.db import run_sql
query= """ 
SELECT
  Año,
  Mes,
  TotalMensual,
  ROW_NUMBER() OVER (
    ORDER BY Año, Mes
  ) AS RowNum
FROM ventas_total_mes;"""

df_customers = run_sql(query)
df_customers 

,Año,Mes,TotalMensual,RowNum
0,2025,1,708878.0,1
1,2025,2,633065.0,2
2,2025,3,742401.0,3
3,2025,4,692481.0,4
4,2025,5,710675.0,5
5,2025,6,729443.0,6
6,2025,7,728310.0,7
7,2025,8,738776.0,8
8,2025,9,718757.0,9
9,2025,10,731963.0,10


Esta función ejecuta el procedimiento almacenado llamado "registro_productos", el cual funciona para almacenar de manera simple nuevos productos para vender

In [1]:
from src.db import execute_sql

query = """
CALL registro_productos(
    9998,
    'Chocolinas',
    200,
    5,
    'Low',
    '2024-06-02',
    'Durable',
    'Unknown',
    200
);
"""
execute_sql(query)


Probamos que funciona el procedimiento almacenado llamando a la tabla productos con el nuevo producto "Chocolinas"

In [2]:
from src.db import run_sql
query= """select * from products
where ProductName='Chocolinas';"""
df_customers = run_sql(query)
df_customers 

,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
0,9998,Chocolinas,200.0,5,Low,0 days,Durable,Unknown,200.0


Trigger de seguridad el cual muestra si el calculo realizado de Total Price se encuentra mal realizado, lo que evita errores o cargas fraudulentas. Se puede crear una tabla para almacenar estos registros erroneos o fraudulentos. Donde en el caso de que este mal la función muestra un error que dice 'Error: TotalPrice no coincide con Quantity * Precio - Discount.'


In [5]:
#Vamos a probarlo (caso de que se esta insertando una fila de Sales con el calculo del Total Price)
from src.db import execute_sql

query = """INSERT INTO sales (
    SalesID, 
    SalesPersonID, 
    CustomerID, 
    ProductID, 
    Quantity, 
    Discount, 
    TotalPrice,            
    SalesDate, 
    TransactionNumber
)
VALUES (
    99999, 
    1, 
    33133, 
    12, 
    2, 
    0.1,
    21.6,                
    '2025-03-15 12:00:00', 
    'TX-TEST-0001'
);"""